# 6장 데이터 로딩과 저장, 파일 형식

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
pd.options.display.max_rows = 10
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [4]:
!cat examples/ex1.csv
 # !쓸시 이 셀에서 보인다.
df = pd.read_csv("examples/ex1.csv") #데이터프레임에 리턴
df.head(2)

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world


    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [12]:
!cat examples/ex2.csv

names = ["a", "b", "c", "d", "message"]
data = pd.read_csv("examples/ex2.csv", names=names)
data['message']

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


0    hello
1    world
2      foo
Name: message, dtype: object

    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [9]:
#!cat examples/csv_mindex.csv

parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
#parsed
parsed.columns

Index(['value1', 'value2'], dtype='object')

    4) 여러 개의 공백으로 필드를 구분한 파일 
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [20]:
!cat examples/ex3.txt

result = pd.read_csv("examples/ex3.txt", sep="\s+")
result

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트

In [22]:
#!cat examples/ex4.csv"

pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    6) **누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력** 
        * na_values 인수 - NA 값으로 처리할 값들의 목록

In [10]:
# !cat "examples/ex5.csv"

result = pd.read_csv("examples/ex5.csv")
result
# 1) 누락된 값을 확인 : isna(),isnull():True,isnot(),notna():결측치 아닌것을 True함수 이용
pd.isna(result)

# # 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("examples/ex5.csv", na_values=sentinels)
print(result)
result.isna()

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,True,False,False,True,False,False
2,False,False,False,False,False,True


    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 읽을 파일의 크기

In [34]:
#pd.options.display.max_rows = 10 //행갯수 제한 10개만
# result = pd.read_csv("examples/ex6.csv")
# result
result = pd.read_csv("examples/ex6.csv", nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


### [예제]  ex6.csv 파일의 "key" 컬럼의 값들에 대한 빈도수을 계산하기

In [60]:
#!cat examples/ex6.csv
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
chunker
tot = Series(data=[],dtype='int64')
#"key" 컬럼의 값들에 대한 누적에 합
tot = pd.Series([], dtype='int64')
for piece in chunker: #10번 반복수행(이유는 파일 갯수는 10000개인데한번에 1000개씩 불러오니 
   #print(piece["key"].value_counts())
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot
#값들을 정렬
tot = tot.sort_values(ascending=False)
tot
# tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

In [53]:
# 시리즈 간 연산
from pandas import Series
tot = Series(data=[],dtype='int64')
tot1=Series(data=[1,2,3])
# tot.add(tot1)
# 누적에 합
for i in range(3):
    tot=tot.add(tot1,fill_value =0)
tot

0    3.0
1    6.0
2    9.0
dtype: float64

## 1.2. csv 텍스트 파일 저장하기
* 판다스에서 CSV 파일을 저장하기: df.to_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


In [68]:
# csv 파일로 저장
# 1. 데이터프레임 준비

data = pd.read_csv("examples/ex5.csv") #원보에 대한 데이터프레임.
data
data1 = data[['a','b','d']] # 컬럼 인덱싱 이용 데이터 추출
print(data1)
# 2. 데이터프레임을 파일로 저장.
data1.to_csv("examples/out.csv") #어느위치에 어떻게 저장하는지 작성


   a   b   d
0  1   2   4
1  5   6   8
2  9  10  12


In [10]:
data1.to_csv("examples/out.csv")
!cat examples/out.csv

,a,b,d
0,1,2,4
1,5,6,8
2,9,10,12


In [12]:
data.to_csv("examples/out.csv", sep="|") # 구분자 사용
data1.to_csv("examples/out.csv", na_rep = "NULL") #누락된 값을 원하는 값으로 지정
data.to_csv("examples/out.csv", index=False, header=False)
data.to_csv("examples/out.csv", index=False, columns=["a", "b", "c"])


# 2. JSON  데이터
* JSON(JavaScript Object Notation)은 텍스트 데이터를 저장하고 전송하기 위한 일반적인 형식
* 주로 웹에서 데이터를 전송하거나 저장하는 데 사용
* JSON 파일은 일반적으로 텍스트 형식으로 작성되며, 사람과 기계 모두 이해하기 쉽다.
* JSON 파일은 키-값 쌍의 모음으로 구성
  * 키-값 쌍은 객체(object)라고도 하며, 중괄호 {} 로 묶는다.
  * 배열(array)은 대괄호 [] 안에 값의 목록을 나열하여 정의.

## 2.1 JSON 데이터 읽기/저장
    * pandas의 to_json() -> 데이터프레임을 JSON 파일로 저
    * read_json() -> JSON 파일에서 데이터프레임을 읽어오기

### [예제]  pandas의 to_json() 및 read_json() 메서드를 사용하여 데이터프레임을 JSON 파일로 저장하고 읽기

In [13]:
# 1. pandas 라이브러리 임포트

import pandas as pd

# 2. JSON 파일로 저장할 데이터프레임 생성
data = {
    "name": ["John", "Alice", "Bob"],
    "age": [30, 25, 35],
    "city": ["New York", "Los Angeles", "Chicago"]
}
df = pd.DataFrame(data)

# 3. 데이터프레임을 JSON 파일로 저장
df.to_json("examples/data.json")

# JSON 파일을 읽어와 데이터프레임으로 변환
df_read = pd.read_json("examples/data.json")

print("Original DataFrame:")
print(df)

print("\nDataFrame from JSON file:")
print(df_read)


Original DataFrame:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago

DataFrame from JSON file:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago


# 3. HTML 데이터
## 3.1 웹 스크래핑 - read_html()

* read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


#### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서의 테이블 데이터 읽어오기
    * 연도별로 부도 은행의 개수을 계산하시오. 

In [11]:
import pandas as pd

#tables = pd.read_html("examples/fdic_failed_bank_list.html") #제공안할시 문제
#1. 정적웹페이지 데이터 읽어오기
tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
type(tables)
len(tables) #리스트 안에 원소는 하나
#print(tables)
failures = tables[0] #인덱싱 한것.
failures
type(failures)
failures.head()

[                         Bank NameBank       CityCity StateSt  CertCert  \
0                        Citizens Bank       Sac City      IA      8758   
1             Heartland Tri-State Bank        Elkhart      KS     25851   
2                  First Republic Bank  San Francisco      CA     59017   
3                       Signature Bank       New York      NY     57053   
4                  Silicon Valley Bank    Santa Clara      CA     24735   
..                                 ...            ...     ...       ...   
563                 Superior Bank, FSB       Hinsdale      IL     32646   
564                Malta National Bank          Malta      OH      6629   
565    First Alliance Bank & Trust Co.     Manchester      NH     34264   
566  National State Bank of Metropolis     Metropolis      IL      3815   
567                   Bank of Honolulu       Honolulu      HI     21029   

                 Acquiring InstitutionAI Closing DateClosing  FundFund  
0              Iowa Trust

,Bank NameBank,CityCity,StateSt,CertCert,Acquiring InstitutionAI,Closing DateClosing,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543
3,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.","March 12, 2023",10540
4,Silicon Valley Bank,Santa Clara,CA,24735,First–Citizens Bank & Trust Company,"March 10, 2023",10539


In [77]:
# 2. 연도별로 부도 은행 개수 : DataFrame 가공
# 1) 컬럼명 확인/수정하기/확인
failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert',
       'AcquiringInstitutionAI', 'ClosingDate', 'FundFund'] #간결히 수정.
failures.head(3)

,BankNameBank,CityCity,StateSt,CertCert,AcquiringInstitutionAI,ClosingDate,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543


In [80]:
# 2) 데이터 변환 : 문자열 object-> datetime
close_timestamps = pd.to_datetime(failures["ClosingDate"]) #칼럼 인덱싱 이용해 시리즈로 추출
close_timestamps
#3) 년도별 데이터 개수 추출
close_timestamps.dt.year.value_counts()

ClosingDate
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2019      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 19, dtype: int64

#### [문제] : 네이버 주식 데이터을 스크래핑하여 csv파일로 저장하기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [82]:
!pip install html5lib  lxml

  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached lxml-5.2.1-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached lxml-5.2.1-cp312-cp312-win_amd64.whl (3.8 MB)


In [92]:
import pandas as pd
#1) 웹페이지 읽어오기.
# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver",encoding='cp949')
df_list
type(df_list)
df_list[1] #인덱싱
#데이터프레임 확인
for df in df_list:
    print("\n",df)



      0      1          2        3          4       5
0  거래량   매수호가   거래대금(백만)  시가총액(억)    영업이익(억)  PER(배)
1   시가   매도호가      전일거래량  자산총계(억)    영업이익증가율  ROE(%)
2   고가  매수총잔량      외국인비율  부채총계(억)   당기순이익(억)  ROA(%)
3   저가  매도총잔량  상장주식수(천주)   매출액(억)   주당순이익(원)  PBR(배)
4  NaN    NaN        NaN   매출액증가율  보통주배당금(원)  유보율(%)

        N       종목명       현재가     전일비     등락률     액면가       시가총액      상장주식수  \
0    NaN       NaN       NaN     NaN     NaN     NaN        NaN        NaN   
1    1.0      삼성전자   85300.0  1200.0  +1.43%   100.0  5092225.0  5969783.0   
2    2.0    SK하이닉스  188000.0  8800.0  +4.91%  5000.0  1368644.0   728002.0   
3    3.0  LG에너지솔루션  379500.0  3500.0  +0.93%   500.0   888030.0   234000.0   
4    4.0  삼성바이오로직스  817000.0  5000.0  +0.62%  2500.0   581492.0    71174.0   
..   ...       ...       ...     ...     ...     ...        ...        ...   
76  49.0      한화오션   25850.0  1900.0  -6.85%  5000.0    79194.0   306359.0   
77  50.0      대한항공   21250.0   250.0  -1.16%  5000.0  

In [101]:
# 2) 데이터프레임 가공:csv파일로 저장.
df = df_list[1]
type(df)
df.head(10)
# 1) 데이터프레임 탐색 : 종목명 컬럼명으로 인덱싱 -Nan 처리
df['종목명']
# 2) 조건색인
cond = df['종목명'].notnull() #불리언 시리즈.
data = df[cond] # 불리언색인 이용해 필터링.
data
data.head(10)
# 3) 데이터프레임 저장.
data.to_csv("examples/Naver_kospi.csv") #가공완료시 저장.

## 3.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용
  * requests.get()
  * requests.get().status_code
  * requests.get().text


### [예제] 패키지 requests을 활용하여 웹페이지 json 데이터을 데이터프레임으로 변환하기

In [3]:
!pip install requests

In [120]:
# 1. requests 모듈 설치하기
# 2. requests 모듈 임포트하기
import requests
import pandas as pd
# 1. 타겟 웹사이트 접속
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url) #응답을 하는것
print(resp.status_code) #접속이 되었나 확인
# 2. 웹페이지 콘텐층 읽어오기
resp.text # 문자열로 반환을 해준다.
#pd.read_html(url)

# 3. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
data
len(data)
data[0].keys()
# print(type(data))
# data[0]['title']
# data[0].keys() # 키 정보 출력
# # 4. 딕셔너리를 DataFrame으로 생성하기
issues = pd.DataFrame(data, columns=["number", "title",
                                      "labels", "state"])
issues.head()


200


,number,title,labels,state
0,58146,BUG: incorrect aggregation of dataframe when using UDF with kwarg,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.g...",open
1,58145,REGR: from_records not initializing subclasses properly,"[{'id': 32815646, 'node_id': 'MDU6TGFiZWwzMjgxNTY0Ng==', 'url': 'https:...",open
2,58143,REF: Allow Index._with_infer to also return RangeIndex,"[{'id': 1218227310, 'node_id': 'MDU6TGFiZWwxMjE4MjI3MzEw', 'url': 'http...",open
3,58142,QST: loc returns matrix with one row when index is not strictly unique,"[{'id': 34444536, 'node_id': 'MDU6TGFiZWwzNDQ0NDUzNg==', 'url': 'https:...",open
4,58141,ENH: Consistent naming conventions for string dtype aliases,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg==', 'url': 'https://api.g...",open


### [문제] 네이버 블로그의 키워드 관련 검색 결과 텍스트 데이터 가져오기 - requests 모듈 사용시 한계점

In [122]:
import requests
url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC"

# params = {
#     'pageNo' : 1,
#     'rangeType' : 'ALL',
#     'orderBy' : 'sim',
#     'keyword' : '파이썬'
# }

# response = requests.get('https://section.blog.naver.com/Search/Post.naver?', params=params)
response = requests.get(url)
print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

200
https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC




<!DOCTYPE html>
<html lang="ko">
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <base href="/home" />
    <meta name="robots" content="noindex,nofollow"/>
    <meta name="referrer" content="always"/>
    <meta name="format-detection" content="telephone=no">
    <link rel="shortcut icon" type="image/x-icon" href="https://section.blog.naver.com/favicon.ico?3"/>
    




 

	
		<meta property="og:title" content="파이썬 : 네이버 블로그"/>

		
			
			
			<meta property="og:image" content="https://blogimgs.pstatic.net/nblog/mylog/post/og_default_image_160610.png"/>
			
		
        <meta property="og:description" content="당신의 모든 기록을 담는 공간"/>

		
	
	

<meta property="me:feed:serviceId" content="blog" />

    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <title>네이버 블로그</title>

    <link rel="stylesheet" type="text/css" href="htt

## 3.3 웹 스크래핑 - beautifulsoup #6주차
* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 
  * soup.select_one()
  * soup.select()
  * get_text()
  * get_attribute()
* 개발자 도구 사용

### [문제] 네이버 지식iN의 검색어 관련 검색 결과 제목 가져오기

In [2]:
import requests
from bs4 import BeautifulSoup
#타겟 웹사이트 지정.
url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

#웹페이지 로딩
response = requests.get(url)

if response.status_code == 200:
    # beautifulsoup 객체로 변환
    soup = BeautifulSoup(response.text, 'html.parser') #바로 변수 안거치고 넣었다.

    # 제목 한개 선택하여 텍스트 가져오기
    #s_content > div.section > ul > li:nth-child(1) > dl > dt > a
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    #print(title)
    # print(title.get_text()) #html 태그에 있는 택스트르 가져오는것.
    # print(title.get_attribute_list("href"))#태그명까지 안에 지정해야된다.링크를 가져오는것

#     # 제목 여러 개 선택하여 텍스트 리스트 가져오기
#s_content > div.section > ul > li:nth-child(2) > dl > dt > a
#s_content > div.section > ul > li:nth-child(3) > dl > dt > a
    titles = soup.select('#s_content > div.section > ul > li:nth-child(n) > dl > dt > a') #여러개의 컨텐츠 가져올시 select 사용.
    #child 를보면 n으로 작성.여러개 작성할수 있다.
    #print(titles)
    for title in titles:
        print("\n",title.get_text()) #titles에서 문장만 뽑아오기.
        print("\n",title.get_attribute_list("href"))
else : 
    print(response.status_code)




 파이썬이 미래에 도움이 될까요?

 ['https://kin.naver.com/qna/detail.naver?d1id=3&dirId=31302&docId=466334742&qb=7YyM7J207I2s&enc=utf8§ion=kin&rank=1&search_sort=0&spq=0']

 중1 파이썬 독학

 ['https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10402&docId=467785974&qb=7YyM7J207I2s&enc=utf8§ion=kin&rank=2&search_sort=0&spq=0']

 파이썬 사용

 ['https://kin.naver.com/qna/detail.naver?d1id=10&dirId=100507&docId=467268227&qb=7YyM7J207I2s&enc=utf8§ion=kin&rank=3&search_sort=0&spq=0']

 맥북 파이썬 idel

 ['https://kin.naver.com/qna/detail.naver?d1id=1&dirId=101&docId=464461438&qb=7YyM7J207I2s&enc=utf8§ion=kin&rank=4&search_sort=0&spq=0']

 파이썬 질문

 ['https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10402&docId=467697482&qb=7YyM7J207I2s&enc=utf8§ion=kin&rank=5&search_sort=0&spq=0']

 파이썬과 자바

 ['https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1040201&docId=466335010&qb=7YyM7J207I2s&enc=utf8§ion=kin&rank=6&search_sort=0&spq=0']

 파이썬 질문...

 ['https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10402&docId=4673

## 3.4 웹 스크래핑 - Selenium을 이용하여 사이트 자동화하기
* selenium 사용법
  1. 시스템의 OS 및 Chrome 브라우저의 버전 확인
  2. Chrome 드라이버 다운로드 및 설치
  3. 크롬 드라이버을 프로젝트 폴더에 저장 - 크롬 드라이버의 설치 경로
     * from selenium import webdriver
     * driver = webdriver.Chrome()
* 동적으로 웹 페이지 정보 추출하기
     * driver.get("웹주소") 
  

### [예제] 패키지 Selenium을 활용하여 특정 웹사이트 자동으로 열기

In [2]:
!pip install selenium

In [3]:
from selenium import webdriver
# Chrome 브라우저 실행
driver = webdriver.Chrome() #크롬 드라이버 실행파일의 위치 -작업디렉터리
# 웹페이지 열기
url = "https://www.google.com" #2. 타겟 웹페이지 로딩
driver.get(url)
#driver.quit() #타겟 브라우저 종료

### [문제] 패키지 Selenium을 활용하여 타겟 웹사이트 자동으로 열기/ 쿼리로 검색하기/ 검색 결과의 제목과 url을 자동으로 추출하여 저장하는 web crawler 만들기
    
    * 사전 준비 작업 : 타겟 웹사이트을 구글 개발도구을 이용하여 탐색하기
    * from selenium.webdriver.common.by import By
    * from selenium.webdriver.common.keys import Keys
    

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By #요소 찾아 정의하기에 사용
from selenium.webdriver.common.keys import Keys #검색어 이용해 검색할때 사용
import time # 파이썬에서 제공하는 내장함수

# Chrome 브라우저 실행
driver = webdriver.Chrome()

# Google 홈페이지 열기
url = "https://www.google.com"
driver.get(url)

# 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(2)

#클로러 하는 내용을 코딩.
try:
    # 검색 상자 요소 찾기
    search_box = driver.find_element(By.NAME, "q") #하나만 요소 찾을시 s안붙여도됨.
    # elemnts ->여러게.,앞 인수는 그대로 쓰고,뒤는 요소 이름이 뭐로 정의 되었는지 찾는다.
    
    # 검색어 입력
    search_query = input("검색키 입력: ") #검색어는 지정해줌,대소문자 구별x
    search_box.send_keys(search_query)
    #이 검색어 이용해 서버에 전달.
    
    # Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN)
    #여기까지 해야된다,엔터키를 입력해야 하기에.

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(15)

    # 검색 결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g") #이 안에 해당하는 것들을 가져오는거.
    print(search_results[0])
    
    #첫번째 제목과 링크 url 추출
    # title = search_results[0].find_element(By.CSS_SELECTOR, "h3").text #.text로 제목 가져오기
    # url =  search_results[0].find_element(By.CSS_SELECTOR, "a").get_attribute("href") #속성 가져오기.
    # print("제목:",title)
    # print("URL:", url)
    
    # 검색 결과의 제목과 URL 인쇄
    for result in search_results:
        title = result.find_element(By.CSS_SELECTOR, "h3").text
        url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        print("제목:", title)
        print("URL:", url)
        print()

except Exception as e:
    print("오류가 발생했습니다:", e)
    # exit()

finally:
    # 브라우저 창 닫기
    driver.quit()                                                                                                        


검색키 입력:  openai


<selenium.webdriver.remote.webelement.WebElement (session="8b5f5797d5679650dd9c039ddb89a6d5", element="f.1AF909A7C72200DCA9282484ED9F2DFC.d.8964117E084A632BEF9DB2B0421EA582.e.1397")>
제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://namu.wiki/w/OpenAI

제목: OpenAI
URL: https://en.wikipedia.org/wiki/OpenAI

제목: 오픈AI - 위키백과, 우리 모두의 백과사전
URL: https://ko.wikipedia.org/wiki/%EC%98%A4%ED%94%88AI

제목: OpenAI
URL: https://www.youtube.com/@OpenAI

제목: OpenAI
URL: https://twitter.com/OpenAI

제목: 설명
URL: https://www.google.com/search?q=openai&sca_esv=9d84270ee46763bd&sca_upv=1&source=hp&ei=YXMaZvqrBMmo1e8Pm8azuAk&iflsig=ANes7DEAAAAAZhqBcS-KqFp8JPLHckDbA4y_yDw5QRRc&udm=&ved=0ahUKEwi64K_Ykb-FAxVJVPUHHRvjDJcQ4dUDCA8&uact=5&oq=openai&gs_lp=Egdnd3Mtd2l6IgZvcGVuYWlIWVAAWBtwAHgAkAEAmAEAoAEAqgEAuAEDyAEA-AEBmAIAoAIAmAMAkgcAoAcA&sclient=gws-wiz#



### [문제] Selenium을 이용하여 네이버 증권 웹페이지 열기/ 삼성전자 일별 시세 HTML가져오기/ 10페이지 내역 테이블 데이터 찾기/검색 결과를 데이터프레임으로 변환/저장하기
* 타겟 사이트: https://finance.naver.com/item/sise.naver?code=005930* 
* https://finance.naver.com/item/sise_day.naver?code=005930&page=4
* https://finance.naver.com/item/sise_day.naver?code=005930&page="

In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd #데이터 프레임 사용
import time

In [24]:
#1. 크롬 드라이버 실행
driver= webdriver.Chrome()

# 2페이지 로딩 시간
time.sleep(2)

# 3. 크롤링 사이트 : 삼성전자의 일별 시세 페이지 URL 찾기
samsung_url ="https://finance.naver.com/item/sise_day.naver?code=005930&page=" #base url
# code= 부가 파라매터를 넘기는 부고 &로 파라매터를 구분
# 4. 첫 페이지부터 10 페이지까지 테이블 담기
samsung_daily_kospi = [] # 리스트 선언

#try: #만약 finally 해제시에는 넘길때마다 문제 가능성 존재 그래서 밖에 생성.
    
for num in range(1,11):
    full_url = samsung_url + str(num)
    # 4.1 해당 페이지 요청
    driver.get(full_url)

    try:
        # 4.2 페이지의 HTML 가져오기
        html = driver.page_source #이속성으로 html가져옴
        df = pd.read_html(html) # 테이블 구조 -> 데이터프레임
        print(type(df))
        print(df)
        # 4.3 결측치 처리 - 조건 색인을 이용한 불리언 인덱싱
        # 조건 색인 
        #cond = df[0]['날짜'].notnull() #null에 대한테이블을 boolean을 한다.
        # df[1]할시 인덱스 에러가 나온다.
        # samsung_day = df[0][cond]
        # # ## 리스트에 담기
        # samsung_daily_kospi.append(samsung_day) #리스트 순서대로 담기

    except Exception as e:
        print("Network Error",e)

    # finally:
    #     driver.quit()

 # 5. 데이터프레임 합치기 #다음주에 더 자세히 배운다.
#print(samsung_daily_kospi[3]) #3개의 테이블 정보를 가져온다.
samsung_daily_kospi_all = pd.concat(samsung_daily_kospi,ignore_index=True)
samsung_daily_kospi_all.head(20)
# 6. 파일로 저장
#samsung_daily_kospi_all.to_csv("samsung_daily_kospi_2024_04_11.csv")

<class 'list'>
[            날짜       종가     전일비       시가       고가       저가         거래량
0          NaN      NaN     NaN      NaN      NaN      NaN         NaN
1   2024.04.12  83700.0   400.0  84700.0  84900.0  83200.0  16963684.0
2   2024.04.11  84100.0   500.0  83200.0  84700.0  82500.0  25538009.0
3   2024.04.09  83600.0   900.0  84500.0  84900.0  83100.0  23725956.0
4   2024.04.08  84500.0     0.0  85200.0  86000.0  84500.0  18953232.0
5   2024.04.05  84500.0   800.0  84500.0  85000.0  83800.0  18883752.0
6          NaN      NaN     NaN      NaN      NaN      NaN         NaN
7          NaN      NaN     NaN      NaN      NaN      NaN         NaN
8          NaN      NaN     NaN      NaN      NaN      NaN         NaN
9   2024.04.04  85300.0  1200.0  85200.0  85500.0  84300.0  25248934.0
10  2024.04.03  84100.0   900.0  84300.0  85000.0  83500.0  30493347.0
11  2024.04.02  85000.0  3000.0  82900.0  85000.0  82900.0  37077944.0
12  2024.04.01  82000.0   400.0  83200.0  83300.0  82000.0  2

MaxRetryError: HTTPConnectionPool(host='localhost', port=14988): Max retries exceeded with url: /session/7aed750bf6e6744e71e12e8f4985035a/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000018115298390>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))